In [18]:
from bs4 import BeautifulSoup
import pandas as pd

column_names = ['Team', 'vs Team', 'Home?', 'Gameweek', 'Shots', 'SoT', 'SiB', 'xG', 'npxG', 'G', 'npG', 'CC', 'xA', 'A', 'xGC', 'GC', 'xCS', 'CS', 'xGI', 'npxGI', 'xP', 'P', 'P vs xP', 'T', 'TPA', 'C 1/3', 'C PA']

with open("page_source.html", "r") as file:
    page_source = file.read()
soup = BeautifulSoup(page_source, "lxml")

In [21]:
from utils.general import get_data_path


season = '2024-25'
df = pd.read_csv(get_data_path(season, "team_gw_data.csv"))
df

,Team,vs Team,Home?,Gameweek,Shots,SoT,SiB,xG,npxG,G,...,CS,xGI,npxGI,xP,P,P vs xP,T,TPA,C 1/3,C PA
0,Arsenal,Wolverhampton Wanderers,True,1,18,6,12,1.2,1.2,2,...,1,2.3,2.3,52.5,70,17.5,567,43,9,11
1,Aston Villa,West Ham,False,1,15,3,10,2.0,2.0,2,...,0,3.4,3.4,38.0,47,9.0,558,21,12,4
2,Brentford,Crystal Palace,True,1,9,5,8,1.5,1.5,2,...,0,2.0,2.0,48.7,47,−1.7,531,11,12,3
3,Wolverhampton Wanderers,Arsenal,False,1,9,3,6,0.5,0.5,0,...,0,0.7,0.7,31.8,20,−11.8,492,16,9,4
4,Brighton,Everton,False,1,10,5,9,1.5,1.5,3,...,1,3.1,3.1,56.3,74,17.7,704,27,13,11
5,Tottenham,Leicester,False,1,15,6,12,1.2,1.2,1,...,0,2.4,2.4,45.1,37,−8.1,830,56,13,11
6,Liverpool,Ipswich,False,1,19,5,13,2.7,2.7,2,...,1,4.8,4.8,65.6,69,3.4,718,46,13,13
7,Leicester,Tottenham,True,1,7,3,6,1.1,1.1,1,...,0,2.0,2.0,41.8,37,−4.8,445,10,3,2
8,Southampton,Newcastle United,False,1,19,3,13,1.9,1.9,0,...,0,3.3,3.3,55.2,22,−33.2,773,46,19,9
9,Ipswich,Liverpool,True,1,7,2,5,0.5,0.5,0,...,0,0.8,0.8,22.7,18,−4.7,495,16,7,5
